In [ ]:
import os
import sys

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import math
from matplotlib.animation import PillowWriter
import matplotlib.patches as patches

In [ ]:
%matplotlib qt

# Population

In [ ]:
def initialize_population(pop_size, xbounds = [0, 1], ybounds = [0, 0.8]):
    '''
    0 : unique ID
    1 : current x coordinate
    2 : current y coordinate
    3 : current heading in x direction
    4 : current heading in y direction
    5 : current speed
    6 : moving
    7 : current state (0=healthy, 1=sick, 2=low_risk)
    
    '''


    population = np.zeros((pop_size, 8))

    
    population[:,0] = [x for x in range(pop_size)]

    
    population[:,1] = np.random.uniform(low = xbounds[0] + 0.05, high = xbounds[1] - 0.05, 
                                        size = (pop_size,))
    population[:,2] = np.random.uniform(low = ybounds[0] + 0.05, high = ybounds[1] - 0.05, 
                                        size=(pop_size,))

    
    population[:,3] = np.random.normal(loc = 0, scale = 1/3, 
                                       size=(pop_size,))
    population[:,4] = np.random.normal(loc = 0, scale = 0.000001, 
                                       size=(pop_size,))

    
    population[:,5] = np.random.normal(0.01, 0.01/3)

    population[:50,7] = 1
    
    
    
    return population


# Movement

In [ ]:
def move(population):
    
    population = update_positions(population)
    
    for i in range(len(population)):
        if population[i,2] <= 0.5 and population[i,2] >= 0.2:
            population[i,6] = 1
            
    return population            

In [ ]:
def update_positions(population):
    
    #x
    population[:,1][(population[:,6] == 1)] = population[:,1][(population[:,6] == 1)] + (population[:,3][(population[:,6] == 1)] * population[:,5][(population[:,6] == 1)]/1.5)
    #y
    population[:,2][(population[:,6] == 1)] = population[:,2][(population[:,6] == 1)] + (population [:,4][(population[:,6] == 1)] * population[:,5][(population[:,6] == 1)]/1.5)
   
    
    return population

In [ ]:
def get_off(population, frame):
    
    get_off = np.random.choice(population[:,0], 175)
    
    for i in range(len(population)):
        if population[i,0] in get_off:
            population[i,6] = 2
    return population

In [ ]:
def update_positions1(population):
    #x&y heading
    population[:,3][(population[:,6] == 2)] = 0.5 - population[:,1][(population[:,6] == 2)]
    population[:,4][(population[:,6] == 2)] = 0.05 - population[:,2][(population[:,6] == 2)]
    
    #x
    population[:,1][(population[:,6] == 2)] = population[:,1][(population[:,6] == 2)] + (population[:,3][(population[:,6] == 2)] * (population[:,5][(population[:,6] == 2)]*5))
    #y
    population[:,2][(population[:,6] == 2)] = population[:,2][(population[:,6] == 2)] + (population [:,4][(population[:,6] == 2)] * (population[:,5][(population[:,6] == 2)]*5))

    return population

In [ ]:
def out_of_bounds(population, xbounds, ybounds):

    shp = population[:,3][(population[:,1] <= xbounds[:,0]) &
                          (population[:,3] < 0)].shape
    population[:,3][(population[:,1] <= xbounds[:,0]) &
                    (population[:,3] < 0)] = np.random.normal(loc = 0.5, 
                                                              scale = 0.5/3,
                                                              size = shp)

    shp = population[:,3][(population[:,1] >= xbounds[:,1]) &
                          (population[:,3] > 0)].shape
    population[:,3][(population[:,1] >= xbounds[:,1]) &
                    (population[:,3] > 0)] = -np.random.normal(loc = 0.5, 
                                                                scale = 0.5/3,
                                                                size = shp)


    shp = population[:,4][(population[:,2] <= ybounds[:,0]) &
                          (population[:,4] < 0)].shape
    population[:,4][(population[:,2] <= ybounds[:,0]) &
                    (population[:,4] < 0)] = np.random.normal(loc = 0.5, 
                                                              scale = 0.5/3,
                                                              size = shp)

    shp = population[:,4][(population[:,2] >= ybounds[:,1]) &
                          (population[:,4] > 0)].shape
    population[:,4][(population[:,2] >= ybounds[:,1]) &
                    (population[:,4] > 0)] = -np.random.normal(loc = 0.5, 
                                                               scale = 0.5/3,
                                                               size = shp)

    return population


In [ ]:
def update_randoms(population, xheading_update_chance=0.02, yheading_update_chance=2,
                   speed_update_chance=0.02):

    #x
    update = np.random.random(size=(pop_size,))
    shp = update[update <= xheading_update_chance].shape
    population[:,3][update <= xheading_update_chance] = np.random.normal(loc = 0, 
                                                       scale = 1/3,
                                                       size = shp)
    #y
    update = np.random.random(size=(pop_size,))
    shp = update[update <= yheading_update_chance].shape
    population[:,4][update <= yheading_update_chance] = np.random.normal(loc = 0, 
                                                       scale = 1/3,
                                                       size = shp)
    #speed
    update = np.random.random(size=(pop_size,))
    shp = update[update <= speed_update_chance].shape
    population[:,5][update <= speed_update_chance] = np.random.normal(loc = 0.01, 
                                                       scale = 0.01/3,
                                                       size = shp)    
    return population

# Infection

In [ ]:
def infect(population,frame):


    infection_range = 0.1
    r = infection_range/1.7
    infected_before = population[population[:,7] == 1]
    healthy_before = population[population[:,7] == 0]
    infection_zone = (r**2) * math.pi


    dist = []
    def distance(a,b):
        for x in range(len(a)): 
            for y in range(len(b)):
                if population[y][7]==0:
                    dis = np.sqrt((a[x][1] - b[y][1])**2 + (a[x][2] - b[y][2])**2) #print the distance

                    dist.append(dis)
                    if dis <= infection_zone:
                        population[y,7] = 2
    distance(infected_before, population)                           
    
    return population


# Visualization

In [ ]:
def update(frame, population,
           xbounds=[0.02, 0.98], ybounds=[0.02, 0.98], 
           visualise=True, infected_plot = []):


    _xbounds = np.array([[xbounds[0] + 0.02, xbounds[1] - 0.02]] * len(population))
    _ybounds = np.array([[ybounds[0] + 0.02, ybounds[1] - 0.02]] * len(population))
    population = out_of_bounds(population, _xbounds, _ybounds)
 

    population = update_randoms(population)
    population = move(population)
    population = update_positions1(population)
    population = infect(population, frame)
    infected_plot.append(len(population[population[:,7] == 2]))
    
        
        
    if frame%30==0:
        population = get_off(population, frame)
    

    if visualise:
        
        ax1.clear()
        
        ax1.scatter(population[:,1][(population[:,6] == 0)], population[:,2][(population[:,6] == 0)], color="gray", s = 2, label='healthy') 
        ax1.scatter(population[:,1][(population[:,6] == 1)], population[:,2][(population[:,6] == 1)], color="gray", s = 2, label='healthy')
        ax1.scatter(population[:,1][(population[:,6] == 2)], population[:,2][(population[:,6] == 2)], color="gray", s = 2, label='healthy')
        
        ax1.scatter(population[:,1][(population[:,7] == 0)], population[:,2][(population[:,7] == 0)], color="gray", s = 2, label='healthy')
        ax1.scatter(population[:,1][(population[:,7] == 1)], population[:,2][(population[:,7] == 1)], color="red", s = 2, label='healthy')
        ax1.scatter(population[:,1][(population[:,7] == 2)], population[:,2][(population[:,7] == 2)], color="orange", s = 2, label='healthy')
        
            
        rec=patches.Rectangle((0.42,0.01), 0.15, 0.15, color="r", fill=False)
        ax1.add_patch(rec)
        rec1=patches.Rectangle((0.425,0.015), 0.14, 0.14, color="w", fill=True)
        ax1.add_patch(rec1)
        
        ax1.text(xbounds[0], 
                 ybounds[1]-0.1, 
                 'timestep: %i' %(frame), fontsize=6)
        
        plt.axis([0,1,0,0.8])
        plt.tick_params(bottom=False, labelbottom=False,
                        left=False, labelleft=False)
        
        ax2.set_ylim(0, pop_size + 100)
        ax2.plot(infected_plot, color='gray')
        
        plt.show()

In [ ]:
if __name__ == '__main__':

    #simulation parameters
    pop_size = 3600
    simulation_steps = 540   
    xbounds = [0, 1] 
    ybounds = [0, 0.8]
    
    fig = plt.figure(figsize=(5,5))
    spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=[5,3])
    
    ax2 = fig.add_subplot(spec[1,0])
    ax2.set_title('number of infected')
    ax2.set_xlim(0, simulation_steps)
    ax2.set_ylim(0, pop_size/2)
    
    population = initialize_population(pop_size)

    ax1 = fig.add_subplot(spec[0,0])
    plt.title('infection simulation')
    plt.xlim(xbounds[0], xbounds[1])
    plt.ylim(ybounds[0], ybounds[1])



    if not os.path.exists('render/'):
        os.makedirs('render/')
           
    

    animation = FuncAnimation(fig, update, fargs = (population,), frames = simulation_steps, interval = 33)
    plt.show()
    
    #animation.save("mart2.gif", writer=PillowWriter(fps=24))
    

    